#Assignment #6

Vision Transformer

---


**SUBMISSION INSTRUCTIONS**

First make a copy of this colab file and then solve the assignment and upload your final notebook on github.

Before uploading your downloaded notebook, RENAME the file as rollno_name.ipynb

Submission Deadline : 31/01/2026 Saturday EOD i.e before 11:59 PM

The deadline is strict, Late submissions will incur penalty

Note that you have to upload your solution on the github page of the project Vision Transformer and **under Assignment 6**

And remember to keep title of your pull request to be ViT_name_rollno_assgn6

Github Submission repo -
https://github.com/electricalengineersiitk/Winter-projects-25-26/tree/main/Vision%20transformer/Assignment%206

# Part A - Data and tokens

Q1. Build a tiny toy dataset with pandas
Create a pandas DataFrame with columns text and label.
- Include at least 12 short sentences (3-10 words each).
- The label is 0/1 (e.g., positive vs negative sentiment).
- Shuffle rows and split into train/test (80/20) using a fixed random seed.
Return: df_train, df_test.

In [1]:
import pandas as pd

def make_toy_dataset(seed: int = 42):
    """Return df_train, df_test with columns: text (str), label (int)."""

    # Step 1: create raw data
    data = {
        "text": [
            "I love this movie",
            "This film was amazing",
            "What a great experience",
            "I enjoyed the story",
            "The acting was very good",
            "Absolutely fantastic direction",
            "I hate this movie",
            "This film was terrible",
            "What a bad experience",
            "I disliked the story",
            "The acting was very poor",
            "Absolutely boring direction"
        ],
        "label": [
            1, 1, 1, 1, 1, 1,   # positive
            0, 0, 0, 0, 0, 0    # negative
        ]
    }

    # Step 2: create DataFrame
    df = pd.DataFrame(data)

    # Step 3: shuffle the DataFrame
    df = df.sample(frac=1, random_state=seed).reset_index(drop=True)

    # Step 4: split into train and test (80/20)
    split_idx = int(0.8 * len(df))
    df_train = df.iloc[:split_idx].reset_index(drop=True)
    df_test = df.iloc[split_idx:].reset_index(drop=True)

    return df_train, df_test


Q2. Clean and tokenize text

Implement a basic cleaner: lowercase, strip, replace multiple spaces with one, and remove punctuation
(.,!?;:).
Tokenize by whitespace.
Add a new column tokens that stores a list of tokens per row.
Return the updated DataFrame.

In [2]:
import re
import pandas as pd

def clean_text(s: str) -> str:

    s = s.lower()

    s = re.sub(r"[.,!?;:]", "", s)

    s = re.sub(r"\s+", " ", s)

    s = s.strip()

    return s


def add_tokens_column(df: pd.DataFrame) -> pd.DataFrame:

    df = df.copy()

    tokens_list = []

    for text in df["text"]:
        cleaned_text = clean_text(text)
        tokens = cleaned_text.split()
        tokens_list.append(tokens)

    df["tokens"] = tokens_list
    return df


Q3. Build a vocabulary + token/id mappings

Build token2id and id2token using the training tokens.
Include special tokens: [PAD], [UNK], [BOS], [EOS] at the beginning.
Add tokens that occur at least min_freq times.
Return: token2id (dict), id2token (list).

In [3]:
from collections import Counter
from typing import Dict, List

SPECIALS = ['[PAD]', '[UNK]', '[BOS]', '[EOS]']

def build_vocab(list_of_token_lists, min_freq: int = 1):
    # Step 1: count token frequencies
    counter = Counter()
    for tokens in list_of_token_lists:
        for token in tokens:
            counter[token] += 1

    # Step 2: start id2token with special tokens
    id2token = []
    for sp in SPECIALS:
        id2token.append(sp)

    # Step 3: add tokens that meet min_freq
    for token, freq in counter.items():
        if freq >= min_freq:
            id2token.append(token)

    # Step 4: build token2id mapping
    token2id = {}
    for idx, token in enumerate(id2token):
        token2id[token] = idx

    return token2id, id2token


Q4. Convert tokens to ids + pad to a batch

Implement tokens_to_ids for one sequence.
Implement pad_batch that takes a list of id sequences and returns:
- X: int array (B,T) padded with pad_id
- pad_mask: bool array (B,T) where True means 'real token' and False means padding

In [4]:
import numpy as np

def tokens_to_ids(tokens, token2id, unk_token='[UNK]'):
    ids = []

    for tok in tokens:
        if tok in token2id:
            ids.append(token2id[tok])
        else:
            ids.append(token2id[unk_token])

    return ids


def pad_batch(id_seqs, pad_id: int):

    B = len(id_seqs)

    max_len = 0
    for seq in id_seqs:
        if len(seq) > max_len:
            max_len = len(seq)

    X = np.zeros((B, max_len), dtype=int)
    pad_mask = np.zeros((B, max_len), dtype=bool)

    for i in range(B):
        seq = id_seqs[i]
        for j in range(len(seq)):
            X[i][j] = seq[j]
            pad_mask[i][j] = True

        for j in range(len(seq), max_len):
            X[i][j] = pad_id
            pad_mask[i][j] = False

    return X, pad_mask



#Part B - Core Transformer math

Q5. Embedding lookup

Implement an embedding table E of shape (V,D) initialized from a normal distribution (mean 0, std 0.02).
Given token ids X (B,T), return embeddings of shape (B,T,D) using NumPy indexing.


In [5]:
import numpy as np

def init_embeddings(vocab_size: int, d_model: int, seed: int = 0):
    # set random seed for reproducibility
    np.random.seed(seed)

    # initialize embedding matrix from normal distribution
    E = np.random.normal(
        loc=0.0,
        scale=0.02,
        size=(vocab_size, d_model)
    )

    return E


def embed(X: np.ndarray, E: np.ndarray):
    """X: (B,T) int, E: (V,D) -> out: (B,T,D)."""

    # numpy indexing automatically does the lookup
    out = E[X]

    return out


Q6. Sinusoidal positional encoding

Implement the classic sinusoidal positional encoding PE of shape (T,D).
Then add it to token embeddings (B,T,D).
Make sure your implementation works for both even and odd D.

In [6]:
import numpy as np

def sinusoidal_positional_encoding(T: int, D: int):
    """Return PE: (T,D)."""

    PE = np.zeros((T, D))

    for pos in range(T):
        for i in range(D):

            div_term = np.power(10000, (2 * (i // 2)) / D)

            if i % 2 == 0:
                PE[pos, i] = np.sin(pos / div_term)
            else:
                PE[pos, i] = np.cos(pos / div_term)

    return PE


def add_positional_encoding(X_emb: np.ndarray, PE: np.ndarray):
    """X_emb: (B,T,D), PE: (T,D) -> (B,T,D)."""

    X_emb_pe = X_emb + PE

    return X_emb_pe


Q7. Scaled dot-product attention with masking

Implement scaled dot-product attention:
Attention(Q,K,V) = softmax((Q @ K^T) / sqrt(dk) + mask) @ V
Inputs: Q,K,V are (B,H,T,Dh). Mask is boolean broadcastable to (B,H,T,T) where False means 'mask out'.
Requirements:
- Use a numerically stable softmax (subtract max).
- Convert boolean mask to large negative values before softmax.
Return: context (B,H,T,Dh) and attention weights (B,H,T,T).

In [7]:
import numpy as np

def softmax(x: np.ndarray, axis: int = -1):
    # subtract max for numerical stability
    x_max = np.max(x, axis=axis, keepdims=True)
    x_exp = np.exp(x - x_max)

    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    y = x_exp / x_sum

    return y


def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Q, K, V: (B, H, T, Dh)
    mask: boolean, broadcastable to (B, H, T, T)
    """

    B, H, T, Dh = Q.shape

    scores = np.matmul(Q, K.transpose(0, 1, 3, 2))

    scores = scores / np.sqrt(Dh)

    if mask is not None:

        scores = scores + (~mask) * (-1e9)


    attn = softmax(scores, axis=-1)

    context = np.matmul(attn, V)

    return context, attn


Q8. Multi-head self-attention (MHA)

Implement multi-head self-attention for input X (B,T,D).
- Project to Q,K,V using weight matrices Wq,Wk,Wv each (D,D).
- Reshape/split into heads -> (B,H,T,Dh) where Dh=D/H.
- Apply scaled dot-product attention with a pad mask (B,T) (broadcast it appropriately).
- Concatenate heads and apply output projection Wo (D,D).
Return: out (B,T,D) and attention weights (B,H,T,T).

In [8]:
import numpy as np

def linear(x: np.ndarray, W: np.ndarray, b=None):
    # x: (B,T,D), W: (D,D)
    y = np.matmul(x, W)
    if b is not None:
        y = y + b
    return y


def split_heads(x: np.ndarray, n_heads: int):
    """(B,T,D) -> (B,H,T,Dh)"""
    B, T, D = x.shape
    Dh = D // n_heads

    # reshape and transpose
    x = x.reshape(B, T, n_heads, Dh)
    xh = x.transpose(0, 2, 1, 3)

    return xh


def combine_heads(xh: np.ndarray):
    """(B,H,T,Dh) -> (B,T,D)"""
    B, H, T, Dh = xh.shape

    xh = xh.transpose(0, 2, 1, 3)
    x = xh.reshape(B, T, H * Dh)

    return x


def mha_self_attention(X, Wq, Wk, Wv, Wo, n_heads: int, pad_mask=None):
    """
    X: (B,T,D)
    pad_mask: (B,T) where True = real token
    """

    # Step 1: linear projections
    Q = linear(X, Wq)
    K = linear(X, Wk)
    V = linear(X, Wv)

    # Step 2: split into heads
    Qh = split_heads(Q, n_heads)
    Kh = split_heads(K, n_heads)
    Vh = split_heads(V, n_heads)

    # Step 3: build attention mask
    attn_mask = None
    if pad_mask is not None:
        # (B,T) -> (B,1,1,T), broadcastable to (B,H,T,T)
        attn_mask = pad_mask[:, None, None, :]

    # Step 4: scaled dot-product a


Q9. LayerNorm + residual connection

Implement LayerNorm for X (B,T,D) using learnable gamma and beta of shape (D,).
Then implement residual_add_and_norm(Y, X, gamma, beta) that returns LayerNorm(X + Y).

In [9]:
import numpy as np

def layer_norm(X: np.ndarray, gamma: np.ndarray, beta: np.ndarray, eps: float = 1e-5):
    """
    X: (B,T,D)
    gamma, beta: (D,)
    """
    mean = np.mean(X, axis=-1, keepdims=True)

    var = np.var(X, axis=-1, keepdims=True)

    X_norm = (X - mean) / np.sqrt(var + eps)

    Y = X_norm * gamma + beta

    return Y


def residual_add_and_norm(Y: np.ndarray, X: np.ndarray, gamma: np.ndarray, beta: np.ndarray):
    # residual connection
    Z = X + Y

    # apply layer normalization
    Z = layer_norm(Z, gamma, beta)

    return Z


Q10. Position-wise FeedForward network

Implement FFN: FFN(X) = relu(X @ W1 + b1) @ W2 + b2
Shapes: X (B,T,D), W1 (D,Dff), b1 (Dff,), W2 (Dff,D), b2 (D,)
Return: (B,T,D).

In [10]:
import numpy as np

def relu(x: np.ndarray):
    y = np.maximum(0, x)
    return y


def feed_forward(X: np.ndarray, W1: np.ndarray, b1: np.ndarray, W2: np.ndarray, b2: np.ndarray):
    """
    X: (B,T,D)
    W1: (D,Dff), b1: (Dff,)
    W2: (Dff,D), b2: (D,)
    """

    # first linear layer
    hidden = np.matmul(X, W1) + b1

    # apply ReLU
    hidden = relu(hidden)

    # second linear layer
    Y = np.matmul(hidden, W2) + b2

    return Y




# Part C - Putting it together

Q11. One Transformer encoder block (forward)

Implement a single encoder block forward pass:
1) MHA = MultiHeadSelfAttention(X) with pad_mask
2) X1 = LayerNorm(X + MHA)
3) FFN = FeedForward(X1)
4) X2 = LayerNorm(X1 + FFN)
Return X2.
You may pass all parameters explicitly (weights, gamma/beta).

In [11]:
def encoder_block_forward(X, params, n_heads: int, pad_mask=None):
    """
    X: (B,T,D)
    params: dict containing all weights
    """

    # ---- Multi-head self-attention ----
    mha_out, attn = mha_self_attention(
        X,
        params["Wq"],
        params["Wk"],
        params["Wv"],
        params["Wo"],
        n_heads,
        pad_mask=pad_mask
    )

    # ---- Residual + LayerNorm (after MHA) ----
    X1 = residual_add_and_norm(
        mha_out,
        X,
        params["ln1_gamma"],
        params["ln1_beta"]
    )

    # ---- Feed Forward Network ----
    ffn_out = feed_forward(
        X1,
        params["W1"],
        params["b1"],
        params["W2"],
        params["b2"]
    )

    # ---- Residual + LayerNorm (after FFN) ----
    X2 = residual_add_and_norm(
        ffn_out,
        X1,
        params["ln2_gamma"],
        params["ln2_beta"]
    )

    return X2


Q12. Sequence classification head + end-to-end demo

Create an end-to-end forward pass for a tiny classifier:
- Input ids -> embeddings + positional enc
- One encoder block
- Pooling: take the [BOS] position (t=0) as the sequence representation
- Linear head: logits = h0 @ Wcls + bcls with Wcls (D,2), bcls (2,)
- Softmax to probabilities
Write predict_proba that takes a batch of texts and returns probs (B,2).
Include simple sanity checks: shapes, probabilities sum to 1, and masking doesn't crash for different
lengths.


In [12]:
def predict_proba(texts, token2id, E, PE, params, Wcls, bcls, n_heads: int):
    """
    texts: list[str]
    Return probs: (B,2)
    """

    # -------- Step 1: clean + tokenize --------
    token_lists = []
    for s in texts:
        s_clean = clean_text(s)
        tokens = s_clean.split()

        # add BOS token at the beginning
        tokens = ['[BOS]'] + tokens
        token_lists.append(tokens)

    # -------- Step 2: tokens -> ids --------
    id_seqs = []
    for tokens in token_lists:
        ids = tokens_to_ids(tokens, token2id)
        id_seqs.append(ids)

    # -------- Step 3: pad batch --------
    pad_id = token2id['[PAD]']
    X_ids, pad_mask = pad_batch(id_seqs, pad_id)
    # X_ids: (B,T), pad_mask: (B,T)

    B, T = X_ids.shape

    # -------- Step 4: embeddings --------
    X_emb = embed(X_ids, E)   # (B,T,D)

    # -------- Step 5: positional encoding --------
    # take only first T positions
    PE_T = PE[:T]
    X_emb = add_positional_encoding(X_emb, PE_T)

    # -------- Step 6: encoder block --------
    X_enc = encoder_block_forward(
        X_emb,
        params,
        n_heads=n_heads,
        pad_mask=pad_mask
    )  # (B,T,D)

    # -------- Step 7: pooling (BOS position) --------
    h0 = X_enc[:, 0, :]   # (B,D)

    # -------- Step 8: classification head --------
    logits = np.matmul(h0, Wcls) + bcls   # (B,2)

    # -------- Step 9: softmax to probabilities --------
    probs = softmax(logits, axis=-1)      # (B,2)

    # -------- Sanity checks --------
    assert probs.shape == (B, 2)
    assert np.allclose(np.sum(probs, axis=1), 1.0, atol=1e-6)

    return probs
